In [1]:
import torchvision.datasets as datasets
import torchvision.models as models
import torch
from torchvision import transforms
from keras.utils import to_categorical
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
from PIL import Image
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torch.utils.data import Subset

2023-05-18 02:12:49.904618: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-18 02:12:50.114359: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-18 02:12:50.824538: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: usr/local/cuda-11.8/lib64
2023-05-18 02:12:50.824697: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cann

Config

In [2]:
config = {
    'batch_size':512,
    'learning_rate':1e-3,
    'weight_decay':5e-4,
    'num_epochs':10,
    'num_classes':10
}

In [3]:
torch.manual_seed(50)
np.random.seed(50)

Load Datasets

In [4]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [5]:
class_nums = {i for i in range(config['num_classes'])}
cifar10_trainset = datasets.CIFAR10(root='/home/user01/cifar10/train/', train=True,
                                    download=False, transform=transform_train)
train_subset_idxs = [idx for idx, label in enumerate(cifar10_trainset.targets) if label in class_nums]
trainloader = torch.utils.data.DataLoader(
    Subset(cifar10_trainset, train_subset_idxs), batch_size=config['batch_size'], shuffle=True, num_workers=7)

In [6]:
cifar10_testset = datasets.CIFAR10(root='/home/user01/cifar10/test/', train=False,
                                   download=False, transform=transform_test)
test_subset_idxs = [idx for idx, label in enumerate(cifar10_testset.targets) if label in class_nums]
testloader = torch.utils.data.DataLoader(
    Subset(cifar10_testset, test_subset_idxs), batch_size=config['batch_size'], shuffle=False, num_workers=7)

In [7]:
for batch, (features, labels) in enumerate(trainloader):
    if batch==0:
        print(features.shape)
        print(labels.shape)
        print(labels)

torch.Size([512, 3, 32, 32])
torch.Size([512])
tensor([8, 6, 0, 5, 6, 8, 3, 7, 9, 1, 7, 3, 6, 4, 5, 4, 7, 9, 2, 0, 8, 9, 0, 1,
        0, 8, 7, 3, 5, 3, 2, 9, 7, 9, 9, 7, 8, 0, 2, 2, 2, 3, 7, 8, 6, 1, 8, 3,
        9, 9, 8, 4, 1, 0, 8, 7, 7, 9, 6, 4, 7, 9, 6, 2, 4, 5, 9, 6, 9, 7, 9, 6,
        4, 7, 2, 0, 0, 1, 4, 2, 9, 4, 2, 7, 4, 7, 3, 7, 2, 4, 8, 2, 0, 3, 9, 5,
        2, 5, 2, 6, 1, 9, 3, 0, 6, 9, 1, 5, 9, 1, 6, 1, 5, 8, 2, 6, 1, 4, 2, 8,
        2, 2, 3, 1, 8, 0, 6, 3, 4, 5, 2, 9, 9, 6, 2, 8, 1, 1, 0, 7, 7, 6, 4, 3,
        0, 1, 7, 2, 5, 4, 4, 5, 0, 4, 1, 0, 0, 0, 9, 3, 0, 3, 7, 9, 4, 7, 0, 1,
        5, 8, 5, 6, 5, 2, 5, 5, 5, 1, 3, 9, 1, 9, 1, 8, 4, 0, 4, 2, 2, 8, 5, 1,
        4, 4, 3, 0, 8, 8, 7, 2, 3, 5, 7, 4, 2, 2, 9, 3, 3, 4, 4, 5, 6, 8, 7, 7,
        7, 7, 2, 3, 3, 1, 8, 3, 4, 4, 5, 9, 6, 2, 3, 5, 9, 7, 8, 4, 8, 3, 1, 1,
        9, 2, 1, 3, 7, 5, 3, 0, 3, 8, 0, 0, 1, 5, 1, 4, 4, 6, 3, 5, 1, 7, 0, 5,
        9, 0, 0, 2, 6, 3, 8, 5, 0, 5, 8, 5, 7, 3, 4, 4, 9, 4, 5, 8, 7, 4,

Model Definition

In [8]:
# get_graph_node_names(model)

In [9]:
new_model = torch.load('cifar100.pth')
# new_model = models.resnet34(weights='ResNet34_Weights.IMAGENET1K_V1')

In [10]:
new_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
for param in new_model.layer1.parameters():
    param.requires_grad = False
for param in new_model.layer2.parameters():
    param.requires_grad = False
for param in new_model.layer3.parameters():
    param.requires_grad = False
for param in new_model.conv1.parameters():
    param.requires_grad = False
for param in new_model.bn1.parameters():
    param.requires_grad = False
for param in new_model.relu.parameters():
    param.requires_grad = False
for param in new_model.maxpool.parameters():
    param.requires_grad = False

In [12]:
# for param in new_model.parameters():
#     param.requires_grad = False

In [13]:
new_model.fc = torch.nn.Linear(in_features=512,out_features=config['num_classes'],bias=True)

In [14]:
for param in new_model.parameters():
    param.requires_grad = True

In [15]:
new_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
# new_model.fc = torch.nn.Linear(in_features=512, out_features=config['num_classes'], bias=True)

In [17]:
# inp = np.random.rand(2,3,32,32)
# new_model(torch.Tensor(inp[:2]))

# Finetuning with CIFAR10

In [18]:
def label_to_out(label):
    out = np.zeros(config['num_classes'])
    out[label] = 1
    return out

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [20]:
new_model = new_model.to(device)

In [21]:
loss_fn = torch.nn.CrossEntropyLoss()

In [22]:
optimizer = torch.optim.Adam(new_model.parameters(), lr=config['learning_rate'],weight_decay=config['weight_decay'])

In [23]:
def output_to_label(out):
    return out.argmax()

In [24]:
def train_loop(dataloader, model, loss_fn, optimizer, epoch_num):
    num_points = len(dataloader.dataset)
    for batch, (features, labels) in enumerate(dataloader):        
        # Compute prediction and loss
        features, labels = features.to(device), labels.to(device)
        pred = model(features)
        loss = loss_fn(pred, labels)
        
        # Backpropagation
        optimizer.zero_grad() # sets gradients of all model parameters to zero
        loss.backward() # calculate the gradients again
        optimizer.step() # w = w - learning_rate * grad(loss)_with_respect_to_w

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(features)
            print(f"Epoch {epoch_num} - loss: {loss:>7f}  [{current:>5d}/{num_points:>5d}]\n")


def test_loop(dataloader, model, loss_fn, epoch_num, name):
    num_points = len(dataloader.dataset)
    sum_test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (features, labels) in enumerate(dataloader):
            features, labels = features.to(device), labels.to(device)
            pred = model(features)
            if batch == 0:
                global tmp_features
                global tmp_labels
                global tmp_pred
                tmp_features = features
                tmp_labels = labels
                tmp_pred = pred
            sum_test_loss += loss_fn(pred, labels).item() # add the current loss to the sum of the losses
            # convert the outputs of the model on the current batch to a numpy array
            pred_lst = list(pred.argmax(axis=1).cpu().numpy())
            # convert the original labels corresponding to the current batch to a numpy array
            true_lst = labels
            # determine the points for which the model is correctly predicting the label (add a 1 for each)
            match_lst = [1 if p==t else 0 for (p, t) in zip(pred_lst, true_lst)] 
            # count how many points are labeled correctly in this batch and add the number to the overall count of the correct labeled points
            correct += sum(match_lst) 
            
    sum_test_loss /= num_points
    correct /= num_points
    print(f"Epoch {epoch_num} - {name} Accuracy: {correct*100}%, Avg loss: {sum_test_loss}\n")

In [25]:
for epoch_num in range(1, config['num_epochs']+1):
    train_loop(trainloader, new_model, loss_fn, optimizer, epoch_num)
    test_loop(testloader, new_model, loss_fn, epoch_num, 'Test')

Epoch 1 - loss: 2.723034  [    0/50000]

Epoch 1 - Test Accuracy: 76.55%, Avg loss: 0.0013861462652683257

Epoch 2 - loss: 0.649873  [    0/50000]

Epoch 2 - Test Accuracy: 79.10000000000001%, Avg loss: 0.0012323650240898133

Epoch 3 - loss: 0.634182  [    0/50000]

Epoch 3 - Test Accuracy: 80.43%, Avg loss: 0.0011551827371120453

Epoch 4 - loss: 0.520214  [    0/50000]

Epoch 4 - Test Accuracy: 81.37%, Avg loss: 0.0010936221063137055

Epoch 5 - loss: 0.417527  [    0/50000]

Epoch 5 - Test Accuracy: 81.92%, Avg loss: 0.0011003970503807067

Epoch 6 - loss: 0.421854  [    0/50000]

Epoch 6 - Test Accuracy: 81.47%, Avg loss: 0.0010709090322256088

Epoch 7 - loss: 0.410383  [    0/50000]

Epoch 7 - Test Accuracy: 82.71%, Avg loss: 0.0010284554302692414

Epoch 8 - loss: 0.448233  [    0/50000]

Epoch 8 - Test Accuracy: 81.44%, Avg loss: 0.0011082372814416885

Epoch 9 - loss: 0.408841  [    0/50000]

Epoch 9 - Test Accuracy: 81.77%, Avg loss: 0.0010916769444942475

Epoch 10 - loss: 0.429544